<a href="https://colab.research.google.com/github/racheopod/piosfinder/blob/master/data_processing/NAIP2019_DownloadSampleofWYNAIP2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exporting 2019 NAIP imagery for Wyoming

Rachel R. Renne

July 2022

This notebook uses Google Earth Engine to access National Agricultural Imagery Program ([NAIP](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-aerial-photography-national-agriculture-imagery-program-naip?qt-science_center_objects=0#qt-science_center_objects)) data in Wyoming.

This files exports two systematically chosen tiles from each NAIP quarter-quad, then exports tiles corresponding to known piosphere and tank locations (based on manual search for piospheres in Google Earth imagery).

## 1. Authenticate to Earth Engine

In [ ]:
!pip install earthengine-api #earth-engine Python API

In [ ]:
!earthengine authenticate

## 2. Software setup

In [ ]:
# Earth Engine Python API
import ee
ee.Initialize()

In [ ]:
# Get geetools
!pip install geetools

### 3. Import imagery and restrict to 2019 in WY

In [ ]:
# Load in states
states = ee.FeatureCollection('TIGER/2018/States')

# Get Wyoming
WY1 = ee.Feature(states.filter('STUSPS == "WY"').first())
wy1 = ee.Algorithms.ProjectionTransform(WY1,'EPSG:26913',0.01)
wy1

In [ ]:
# Load a landsat image and select three bands.
naipyear = ee.ImageCollection('USDA/NAIP/DOQQ')\
    .filterDate('2019-01-01', '2020-1-01')\
    .filterBounds(wy1.geometry())\
    .select('R','G','B','N')

naipyear

In [ ]:
# Check number of images (7407)
imgcount = naipyear.size().getInfo()
print(imgcount)

7421


### 4. Download a selection of cells from a covering grid for each quarter quad

In [ ]:
#Create function to systematically select cells 466 an 1397 from all quarter quads:
def getfirstCell(typicalImage):
  #Create a covering grid for context
  thisGrid = typicalImage.geometry(0.001, 'EPSG:26913').coveringGrid('EPSG:26913',153.6)
  #Count the number of grid cells in each quarter quad
  size1 = thisGrid.size()
  #Grab two evenly spaced grid cells would be 465 and 1396
  thiscell = ee.Feature(thisGrid.toList(size1).get(465))
  return thiscell

def getsecondCell(typicalImage):
  #Create a covering grid for context
  thisGrid = typicalImage.geometry(0.001, 'EPSG:26913').coveringGrid('EPSG:26913',153.6)
  #Count the number of grid cells in each quarter quad
  size1 = thisGrid.size()
  #Grab two evenly spaced grid cells would be 465 and 1396
  thiscell = ee.Feature(thisGrid.toList(size1).get(1396))
  return thiscell

#Create FeatureCollection of those selected cells
selectedCells1 = ee.FeatureCollection(naipyear.map(getfirstCell))
selectedCells2 = ee.FeatureCollection(naipyear.map(getsecondCell))
selectedCells = selectedCells1.merge(selectedCells2)

#Count total cells
totalCells = selectedCells.size()

# Count total number of cells and print out
maxind = totalCells.getInfo()
print(maxind)



14842


Try exporting to Google Clound Storage

In [ ]:
# Designate output bucket
outputBucket = 'wynaip2019'

# Make a list of images (start with 2 to try)
#imagesList = naip2.toList(naip2.size())
imagesList = naipyear.toList(naipyear.size())
# Double check number of images
nimg = imagesList.size().getInfo()
nimg

7421

Create some test code to verify that subsequent functions/loops will work

In [ ]:
# Pull out one image
image = ee.Image(naipyear.toList(1,1083).get(0))

# Get one cell to try
selectedCells1 = getfirstCell(image)
fileName = image.id().getInfo()
fileName = fileName + '_465'
fileName

'm_4110753_sw_13_060_20190712_465'

In [ ]:
# Create empty list of completed images
completed_images = []
completed_images

[]

In [ ]:
# Run loop to export (start may need adjusted to restart process when interrupted)
for i in range(6102, nimg):
    print(i)
    image= ee.Image(imagesList.get(i))

    # Get first & second cells
    selectedCells1 = getfirstCell(image)
    selectedCells2 = getsecondCell(image)
    
    # Set up filename
    imgid = image.id().getInfo()
    completed_images.append(imgid)
    fileName1 = imgid + '_465'
    fileName2 = imgid + '_1396'

    # Get bounds of thisImage
    #thisLinearRing = ee.Geometry(image.get('system:footprint'))
    #thisbounds = thisLinearRing.bounds(0.001, 'EPSG:26913')

    #Process the export for you image into the bucket in GCS
    task = ee.batch.Export.image.toCloudStorage(**{
        'image': image,
        'description': 'TryTwo',
        'crs': 'EPSG:26913',
        'scale': 0.6,
        'fileDimensions': 256,
        'fileNamePrefix': fileName1,
        'region': selectedCells1.geometry(),
        'fileFormat': 'GeoTIFF',
        'maxPixels': 10000000000000,
        'bucket': outputBucket,
        'formatOptions': {'cloudOptimized': True},
        'skipEmptyTiles': True})
    task.start()

    #Process the export for you image into the bucket in GCS
    task2 = ee.batch.Export.image.toCloudStorage(**{
        'image': image,
        'description': 'TryTwo',
        'crs': 'EPSG:26913',
        'scale': 0.6,
        'fileDimensions': 256,
        'fileNamePrefix': fileName2,
        'region': selectedCells2.geometry(),
        'fileFormat': 'GeoTIFF',
        'maxPixels': 10000000000000,
        'bucket': outputBucket,
        'formatOptions': {'cloudOptimized': True},
        'skipEmptyTiles': True})
    task2.start()

In [ ]:
#Create function to systematically select cells 466 an 1397 from all quarter quads
#But account for two different projections
def getfirstCell1(typicalImage):
  thisProjection = typicalImage.select('R').projection()
  #Create a covering grid for context
  thisGrid = typicalImage.geometry(0.001, thisProjection).coveringGrid(thisProjection,153.6)
  #Count the number of grid cells in each quarter quad
  size1 = thisGrid.size()
  #Grab two evenly spaced grid cells would be 465 and 1396
  thiscell = ee.Feature(thisGrid.toList(size1).get(465))
  return thiscell

def getsecondCell1(typicalImage):
  thisProjection = typicalImage.select('R').projection()
  #Create a covering grid for context
  thisGrid = typicalImage.geometry(0.001, thisProjection).coveringGrid(thisProjection,153.6)
  #Count the number of grid cells in each quarter quad
  size1 = thisGrid.size()
  #Grab two evenly spaced grid cells would be 465 and 1396
  thiscell = ee.Feature(thisGrid.toList(size1).get(1396))
  return thiscell

In [ ]:
# Run loop to export (start may need adjusted to restart process when interrupted)
# Add if statement to deal with images in different projections
for i in range(6264, nimg):
    print(i)
    image= ee.Image(imagesList.get(i))
    # Get projection information for NAIP (varies by tile/location)
    thisProjection = image.select('R').projection()

    # Get first & second cells
    selectedCells1 = getfirstCell1(image)
    selectedCells2 = getsecondCell1(image)
    
    # Set up filename
    imgid = image.id().getInfo()
    fileName1 = imgid + '_465'
    fileName2 = imgid + '_1396'

    # Get bounds of thisImage
    #thisLinearRing = ee.Geometry(image.get('system:footprint'))
    #thisbounds = thisLinearRing.bounds(0.001, 'EPSG:26913')

    if thisProjection != 'EPSG:26913':
      #Process the export for you image into the bucket in GCS
      task = ee.batch.Export.image.toCloudStorage(**{
          'image': image,
          'description': 'TryTwo',
          'crs': thisProjection,
          'scale': 0.6,
          'fileDimensions': 256,
          'fileNamePrefix': fileName1,
          'region': selectedCells1.geometry(),
          'fileFormat': 'GeoTIFF',
          'maxPixels': 10000000000000,
          'bucket': outputBucket,
          'formatOptions': {'cloudOptimized': True},
          'skipEmptyTiles': True})
      task.start()

      #Process the export for you image into the bucket in GCS
      task2 = ee.batch.Export.image.toCloudStorage(**{
          'image': image,
          'description': 'TryTwo',
          'crs': thisProjection,
          'scale': 0.6,
          'fileDimensions': 256,
          'fileNamePrefix': fileName2,
          'region': selectedCells2.geometry(),
          'fileFormat': 'GeoTIFF',
          'maxPixels': 10000000000000,
          'bucket': outputBucket,
          'formatOptions': {'cloudOptimized': True},
          'skipEmptyTiles': True})
      task2.start()

### 5. Now download cells with known piospheres

Mosaic naip data for piosphere cell downloads below.

In [ ]:
#Mosaic naipyear
naipmosaic = naipyear.mosaic()

In [ ]:
# Designate output bucket for known
outputBucket1 = 'piospheresnaip2019'

In [ ]:
# Create LIST of points
pts = ee.FeatureCollection('projects/renne-piospheres-cv4ecology/assets/piospheres_20220727')
ptList = pts.toList(pts.size());

In [ ]:
# Function creating new index
def newID(newSysIndex):
    feat = ee.Feature(gridList.get(newSysIndex))
    # format number to string
    indexString = ee.Number(newSysIndex)
    return feat.set('ID', indexString)

# Count points
npts = pts.size().getInfo()
#print(npts)

# Loop through points
for i in range(0, npts):
  print(i)
  thispt = ptList.get(i)
  # print(thispt)
  # transform point
  thispt1 = ee.Feature(thispt).transform('EPSG:26913',0.001)
  # get the relevant image
  thisImage = naipyear.filterBounds(thispt1.geometry()).first()
  # Get image ID
  imgid = thisImage.id().getInfo()
  # Get projection information for NAIP (varies by tile/location)
  thisProjection = thisImage.select('R').projection()
  # Get image ID
  imgid = thisImage.id().getInfo()
  # Create grid
  thisGrid = thisImage.geometry(0.001, thisProjection).coveringGrid(thisProjection,153.6)
  # make a list of length of features
  idList = ee.List.sequence(0,thisGrid.size().subtract(1))
  # featureCollection to a List
  gridList = thisGrid.toList(thisGrid.size())
  # Set new IDs
  assetID = ee.FeatureCollection(idList.map(newID))
  # Create updated feature collection
  updatedGrid = ee.FeatureCollection(assetID)
  # Pull out a cell that contains a feature of interest
  thisCell = ee.Feature(updatedGrid.filterBounds(thispt1.geometry()).first())
  cellnum = ee.Feature(thisCell).get('ID').getInfo()
  fileName = imgid + '_' + str(cellnum)
  #print(str(i) + ': ' + fileName)
  
  #Process the export for you image into the bucket in GCS
  task = ee.batch.Export.image.toCloudStorage(**{
       'image': naipmosaic,
       'description': 'ExportPiosphereCells',
       'crs': thisProjection,
       'scale': 0.6,
       'fileDimensions': 256,
       'fileNamePrefix': fileName,
       'region': thisCell.geometry(),
       'fileFormat': 'GeoTIFF',
       'maxPixels': 10000000000000,
       'bucket': outputBucket1,
       'formatOptions': {'cloudOptimized': True},
       'skipEmptyTiles': True})
  task.start()

In [ ]:
# Loop through points again but using different projection
for i in range(0, npts):
  print(i)
  thispt = ptList.get(i)
  # print(thispt)
  # transform point
  thispt1 = ee.Feature(thispt).transform(' EPSG:26912',0.001)
  # get the relevant image
  thisImage = naipyear.filterBounds(thispt1.geometry()).first()
  # Get image ID
  imgid = thisImage.id().getInfo()
  # Get projection information for NAIP (varies by tile/location)
  thisProjection = thisImage.select('R').projection()
  # Get image ID
  imgid = thisImage.id().getInfo()
  # Create grid
  thisGrid = thisImage.geometry(0.001, thisProjection).coveringGrid(thisProjection,153.6)
  # make a list of length of features
  idList = ee.List.sequence(0,thisGrid.size().subtract(1))
  # featureCollection to a List
  gridList = thisGrid.toList(thisGrid.size())
  # Set new IDs
  assetID = ee.FeatureCollection(idList.map(newID))
  # Create updated feature collection
  updatedGrid = ee.FeatureCollection(assetID)
  # Pull out a cell that contains a feature of interest
  thisCell = ee.Feature(updatedGrid.filterBounds(thispt1.geometry()).first())
  cellnum = ee.Feature(thisCell).get('ID').getInfo()
  fileName = imgid + '_' + str(cellnum)
  #print(str(i) + ': ' + fileName)
  
  #Process the export for you image into the bucket in GCS
  task = ee.batch.Export.image.toCloudStorage(**{
       'image': naipmosaic,
       'description': 'ExportPiosphereCells',
       'crs': thisProjection,
       'scale': 0.6,
       'fileDimensions': 256,
       'fileNamePrefix': fileName,
       'region': thisCell.geometry(),
       'fileFormat': 'GeoTIFF',
       'maxPixels': 10000000000000,
       'bucket': outputBucket1,
       'formatOptions': {'cloudOptimized': True},
       'skipEmptyTiles': True})
  task.start()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27